In [1]:
import streamlit as st

import tempfile
from transformers import pipeline
import requests
import os
from dotenv import load_dotenv

API_TOKEN = os.getenv('wav2vec2_TOKEN')
API_URL = "https://api-inference.huggingface.co/models/facebook/wav2vec2-large-xlsr-53-german"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

audio_path = "./audio/Richard.wav"

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query(audio_path)
output["text"]

c:\Users\cbg-rs\Desktop\IPT_S2T\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'im ersten schritt des prozesses wird das werkzeug in den dafür vorhergesehenen arbeitsplatz gelegt nach dem staaten der anlage wird der arbeitsplatz gereinigt und das nächste werkzeug vom anlagen ophoweter vorbereitet um den nächsten betsch zu verarbeiten'

In [15]:
import whisper

audio_path = "./audio/Richard.wav"

model = whisper.load_model("large")
result = model.transcribe(audio_path, fp16=False)
result["text"]

' Im ersten Schritt des Prozesses wird das Werkzeug in den dafür vorhergesehenen Arbeitsplatz gelegt. Nach dem Starten der Anlage wird der Arbeitsplatz gereinigt und das nächste Werkzeug vom Anlagenoperator vorbereitet, um den nächsten Batch zu verarbeiten.'

In [8]:
API_TOKEN = os.getenv('wav2vec2_TOKEN')
API_URL = "https://api-inference.huggingface.co/models/jonatasgrosman/wav2vec2-large-xlsr-53-german"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

audio_path = "./audio/Richard.wav"

def query(filename):
    with open(filename, "rb") as f:
        data = f.read()
    response = requests.post(API_URL, headers=headers, data=data)
    return response.json()

output = query(audio_path)
output["text"]

{'text': 'im ersten schritt des prozesses wird das werkzeug in den dafür vorhergesehenen arbeitsplatz gelegt nach dem starten der anlage wird der arbeitsplatz gereinigt und das nächste werkzeug vom anlagen operator vorbereitet um den nächsten batch zu verarbeiten'}

In [4]:
import assemblyai as aai

# Replace with your API key
aai.settings.api_key = "3ea47c494d314b1bbe6ab71a71fb4670"

# URL of the file to transcribe
FILE_URL = "audio/Richard.wav"

# You can also transcribe a local file by passing in a file path
# FILE_URL = './path/to/file.mp3'
config = aai.TranscriptionConfig(language_code="de")
transcriber = aai.Transcriber(config=config)
transcript = transcriber.transcribe(FILE_URL)
transcript.text

'Im ersten Schritt des Prozesses wird das Werkzeug in den dafür vorhergesehenen Arbeitsplatz gelegt. Nach dem Starten der Anlage wird der Arbeitsplatz gereinigt und das nächste Werkzeug vom Anlagenoperator vorbereitet, um den nächsten Batch zu verarbeiten.'

In [14]:
def compare_texts(original_text, generated_text):
    original_words = original_text.split()
    generated_words = generated_text.split()
    
    comparison_results = []
    
    for i in range(max(len(original_words), len(generated_words))):
        if i > 0:
            comparison_results.append(" ")
        if i < len(original_words) and i < len(generated_words):
            if original_words[i].lower() != generated_words[i].lower():
                comparison_results.append((generated_words[i], "wrong", "#FFC7CE"))
            else:
                comparison_results.append(original_words[i])
        elif i >= len(original_words):
            comparison_results.append((generated_words[i], "wrong", "#FFC7CE"))
    
    return comparison_results

# Example usage
original_text = "Nach dem Starten der Anlage wird der Arbeitsplatz gereinigt und das nächste Werkzeug vom Anlagenoperator vorbereitet, um den nächsten Batch zu verarbeiten."
generated_text = "Nach dem Starten der Anlage wird der Arbeits platz gereinigt und das nächste Werkzeug vom Anlagen operator vorbereitet, um den nächsten Batch zu verarbeiten."

# Call the function and print the result
comparison_result = compare_texts(original_text, generated_text)
print(comparison_result)

['Nach', ' ', 'dem', ' ', 'Starten', ' ', 'der', ' ', 'Anlage', ' ', 'wird', ' ', 'der', ' ', ('Arbeits', 'wrong', '#FFC7CE'), ' ', ('platz', 'wrong', '#FFC7CE'), ' ', ('gereinigt', 'wrong', '#FFC7CE'), ' ', ('und', 'wrong', '#FFC7CE'), ' ', ('das', 'wrong', '#FFC7CE'), ' ', ('nächste', 'wrong', '#FFC7CE'), ' ', ('Werkzeug', 'wrong', '#FFC7CE'), ' ', ('vom', 'wrong', '#FFC7CE'), ' ', ('Anlagen', 'wrong', '#FFC7CE'), ' ', ('operator', 'wrong', '#FFC7CE'), ' ', ('vorbereitet,', 'wrong', '#FFC7CE'), ' ', ('um', 'wrong', '#FFC7CE'), ' ', ('den', 'wrong', '#FFC7CE'), ' ', ('nächsten', 'wrong', '#FFC7CE'), ' ', ('Batch', 'wrong', '#FFC7CE'), ' ', ('zu', 'wrong', '#FFC7CE'), ' ', ('verarbeiten.', 'wrong', '#FFC7CE')]


In [17]:
def compare_texts_advanced(original_text, generated_text):
    # Convert texts into lowercase word lists for case-insensitive comparison
    original_words = original_text.lower().split()
    generated_words = generated_text.lower().split()

    # Initialize dynamic programming matrix
    m, n = len(original_words), len(generated_words)
    dp = [[0 for j in range(n+1)] for i in range(m+1)]

    # Populate DP table with edit distances
    for i in range(m+1):
        for j in range(n+1):
            if i == 0:
                dp[i][j] = j  # Deletion
            elif j == 0:
                dp[i][j] = i  # Insertion
            elif original_words[i-1] == generated_words[j-1]:
                dp[i][j] = dp[i-1][j-1]  # No change needed
            else:
                dp[i][j] = 1 + min(dp[i-1][j],  # Insertion
                                   dp[i][j-1],  # Deletion
                                   dp[i-1][j-1])  # Substitution

    # Backtrack to find alignment
    i, j = m, n
    alignment = []
    while i > 0 and j > 0:
        if original_words[i-1] == generated_words[j-1]:
            alignment.append((original_words[i-1], "correct"))
            i -= 1
            j -= 1
        elif dp[i][j] == 1 + dp[i-1][j-1]:  # Substitution
            alignment.append((generated_words[j-1], "wrong", "#FFC7CE"))
            i -= 1
            j -= 1
        elif dp[i][j] == 1 + dp[i-1][j]:  # Insertion
            alignment.append((original_words[i-1], "missing", "#FFC7CE"))
            i -= 1
        else:  # Deletion
            alignment.append((generated_words[j-1], "extra", "#FFC7CE"))
            j -= 1

    # Handle remaining words
    while i > 0:
        alignment.append((original_words[i-1], "missing", "#FFC7CE"))
        i -= 1
    while j > 0:
        alignment.append((generated_words[j-1], "extra", "#FFC7CE"))
        j -= 1

    # Reverse alignment to correct order
    alignment.reverse()

    # Generate comparison results with proper spacing
    comparison_results = []
    for i, (word, status, *color) in enumerate(alignment):
        if i > 0:
            comparison_results.append(" ")
        if status == "correct":
            comparison_results.append(word)
        else:
            comparison_results.append((word, status, color[0] if color else "#FFC7CE"))

    return comparison_results

# Example usage with a simple case
original_text = "Nach dem Starten der Anlage wird der Arbeitsplatz gereinigt und das nächste Werkzeug vom Anlagenoperator vorbereitet, um den nächsten Batch zu verarbeiten."
generated_text = "Nach dem Starten der Anlage wird der Arbeits platz gereinigt und das nächste Werkzeug vom Anlagen operator vorbereitet, um den nächsten Batch zu verarbeiten."

print(compare_texts_advanced(original_text, generated_text))

['nach', ' ', 'dem', ' ', 'starten', ' ', 'der', ' ', 'anlage', ' ', 'wird', ' ', 'der', ' ', ('arbeits', 'extra', '#FFC7CE'), ' ', ('platz', 'wrong', '#FFC7CE'), ' ', 'gereinigt', ' ', 'und', ' ', 'das', ' ', 'nächste', ' ', 'werkzeug', ' ', 'vom', ' ', ('anlagen', 'extra', '#FFC7CE'), ' ', ('operator', 'wrong', '#FFC7CE'), ' ', 'vorbereitet,', ' ', 'um', ' ', 'den', ' ', 'nächsten', ' ', 'batch', ' ', 'zu', ' ', 'verarbeiten.']
